<a href="https://colab.research.google.com/github/cedro3/DeZero_generate_japanese_text/blob/master/DeZero_generate_japanese_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

フレームワーク **DeZero** のインストール

In [ ]:
!pip install dezero

形態素分析ライブラリー **janome** のインストール

In [ ]:
!pip install janome

日本語データセット **Nekoクラス**

In [ ]:
import numpy as np
import dezero
from dezero.datasets import Dataset
from dezero.utils import get_file, cache_dir
import zipfile
import re
from janome.tokenizer import Tokenizer

class Neko(Dataset):
    
    def prepare(self):
        url = 'https://www.aozora.gr.jp/cards/000148/files/789_ruby_5639.zip'
        file = get_file(url)  
        data = self.unzip(cache_dir + '/' + '789_ruby_5639.zip')  
        self.text = self.preprocess(cache_dir + '/' + 'wagahaiwa_nekodearu.txt')
        self.wakati = self.keitaiso(self.text)
        self.corpus, self.word_to_id, self.id_to_word = self.process(self.wakati)
        self.data = np.array(self.corpus[:-1])
        self.label = np.array(self.corpus[1:])
    
    def unzip(self, file_path):
        with zipfile.ZipFile(file_path) as existing_zip:
            existing_zip.extractall(cache_dir)
            
    def preprocess(self, file_path):
        binarydata = open(file_path, 'rb').read()
        text = binarydata.decode('shift_jis')        
                   
        text = re.split(r'\-{5,}', text)[2]  # ヘッダの削除
        text = re.split('底本：',text)[0]   # フッタの削除
        text = re.sub('｜', '', text)  # | の削除
        text = re.sub('［.+?］', '', text)  # 入力注の削除
        text = re.sub(r'《.+?》', '', text)  # ルビの削除
        text = re.sub(r'\u3000', '', text)  # 空白の削除
        text = re.sub(r'\r\n', '', text)  # 改行の削除
        text = text[1:]  # 先頭の１文字を削除（調整）
        return text
 
    def keitaiso(self, text):
        t = Tokenizer()
        output = t.tokenize(text, wakati=True)
        return output
     
    def process(self, text):
        # word_to_id, id_to_ward の作成
        word_to_id, id_to_word = {}, {}
        for word in text:
            if word not in word_to_id:
                new_id = len(word_to_id)
                word_to_id[word] = new_id
                id_to_word[new_id] = word

        # corpus の作成
        corpus = np.array([word_to_id[W] for W in text])
        return corpus, word_to_id, id_to_word

データセットの作成 ( **Nekoクラス**のインスタンス化。実行に数十秒ほど掛かります)

In [ ]:
neko = Neko()

作成したデータセットの内容を見てみます。

In [ ]:
print('neko.text = ', neko.text[:50])
print('neko.wakati = ', neko.wakati[:15])
print('neko.corpus = ',neko.corpus[:15])

In [ ]:
print('neko.word_to_id [猫] = ', neko.word_to_id['猫'])
print('neko.id_to_word [6] = ', neko.id_to_word[6])

In [ ]:
print('neko.data = ', neko.data[:15])
print('neko.label = ', neko.label[:15])

In [ ]:
print('length od data = ', len(neko.data))
print('vaocab_size = ', len(neko.word_to_id))

**コード本体**です。１epoch毎に、100単語の文章を生成します。1eopch当たりの処理時間は2分くらいです。

In [ ]:
import numpy as np
import dezero
from dezero import Model
from dezero import SeqDataLoader
import dezero.functions as F
import dezero.layers as L
import random
from dezero import cuda 
import textwrap

max_epoch = 70
batch_size = 30 
vocab_size = len(neko.word_to_id)  
wordvec_size = 650  
hidden_size = 650
bptt_length = 30  

class Lstm_nlp(Model):
    def __init__(self, vocab_size, wordvec_size, hidden_size, out_size):
        super().__init__()
        self.embed = L.EmbedID(vocab_size, wordvec_size)
        self.rnn = L.LSTM(hidden_size)
        self.fc = L.Linear(out_size)

    def reset_state(self):  # 状態リセット
        self.rnn.reset_state()

    def __call__(self, x):  # レイヤの接続内容を記載
        y = self.embed(x) 
        y = self.rnn(y)
        y = self.fc(y)
        return y

model = Lstm_nlp(vocab_size, wordvec_size, hidden_size, vocab_size)  # モデル生成
dataloader = SeqDataLoader(neko, batch_size=batch_size)  # データローダ生成
seqlen = len(neko)
optimizer = dezero.optimizers.Adam().setup(model)  # 最適化手法は Adam

# GPUの有無判定と処理
if dezero.cuda.gpu_enable:  # GPUが有効であれば下記を実行
    dataloader.to_gpu()  # データローダをGPUへ
    model.to_gpu()  # モデルをGPUへ

# 学習ループ
for epoch in range(max_epoch):
    model.reset_state()
    loss, count = 0, 0

    for x, t in dataloader:
        y = model(x)  # 順伝播

        # y は次の単語の出現度合いを表すベクトル(vocab_size次元)。
        # y にsoftmaxを掛け出現確率にしたものとワンホットの次の正解データからロス計算。
        # 但し、入力 t はワンホットベクトルの何番目に1が立っているかを表す数字(整数)。
        loss += F.softmax_cross_entropy_simple(y, t)  
        count += 1

        if count % bptt_length == 0 or count == seqlen:
            model.cleargrads()  # 微分の初期化
            loss.backward()  # 逆伝播
            loss.unchain_backward()  # 計算グラフを遡ってつながりを切る
            optimizer.update()  # 重みの更新
    avg_loss = float(loss.data) / count
    print('| epoch %d | loss %f' % (epoch + 1, avg_loss))

    # 文章生成
    model.reset_state()  # 状態をリセット
    with dezero.no_grad():  # 重みの更新をしない
         text = []
         x = random.randint(0,vocab_size)  # 最初の単語番号をランダムに選ぶ
         while len(text)  < 100:  # 100単語になるまで繰り返す
               x = np.array(int(x))
               y = model(x)  # yは次の単語の出現度合い
               p = F.softmax_simple(y, axis=0)  # softmax を掛けて出現確率にする
               xp = cuda.get_array_module(p)  # GPUがあれば xp=cp なければ xp=np
               sampled = xp.random.choice(len(p.data), size=1, p=p.data)  #　出現確率を考慮して数字(インデックス)を選ぶ
               word = neko.id_to_word[int(sampled)]  # 数字を単語に変換
               text.append(word)  # text に単語を追加
               x = sampled  # sampledを次の入力にする
         text = ''.join(text)
         print(textwrap.fill(text, 60))  # 60文字で改行して表示
    